<a href="https://colab.research.google.com/github/SandipanMajhi/Document-Analysis/blob/main/Mnist_feature_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as datasets
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm

%matplotlib inline

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
device

'cuda'

In [4]:
train_set = datasets.MNIST(root = "./data", train=True, download=True, transform=None)
test_set = datasets.MNIST(root = "./data", train=False, download = True, transform=None)

In [5]:
train_arrays = []
train_labels = []
test_arrays = []
test_labels = []

def pil_to_np(stock_set):
  target_threshold = []
  target_set = []
  target_labels = []
  for i in range(0, len(stock_set)):
    img = np.array(stock_set[i][0], dtype = np.uint8)
    thresh , im_bw = cv2.threshold(img,128,255, cv2.THRESH_BINARY | cv2.THRESH_OTSU) 
    target_threshold.append(thresh)
    target_set.append(np.array(im_bw, dtype = np.uint8))
    target_labels.append(stock_set[i][1])
    # target_labels.append(np.array(stock_set[i][1]))

  return target_threshold, target_set, target_labels

train_thresh, train_arrays, train_labels = pil_to_np(train_set)
test_thresh, test_arrays, test_labels = pil_to_np(test_set)

In [6]:
train_arrays[0].shape

(28, 28)

In [7]:
class Data_Maps():
  def __init__(self, dataset):
    self.dataset = dataset
      
  
  def map_diff(self, cmaps):
    maps = []
    for i in range(len(cmaps)):
      sz = len(cmaps[i])
      temp = cmaps[i][0]
      for j in range(0,len(cmaps[i])):
        cmaps[i][j] = (cmaps[i][j] - cmaps[i][(j+1)%sz])
      cmaps[i][-1] = (cmaps[i][-1] - temp)
    return cmaps

  def left_maps(self, rstep, cstep):
    sz = len(self.dataset)
    left_maps = []
    for i in range(sz):
      lmap = []
      for row in range(0,self.dataset[i].shape[0],rstep):
        count = 0
        for col in range(0,self.dataset[i].shape[1],cstep):
          if self.dataset[i][row][col] == 0:
            count = count + 1
          else:
            break
        lmap.append(count)
      left_maps.append(lmap)
    return left_maps

  def right_maps(self, rstep, cstep):
    sz = len(self.dataset)
    right_maps = []
    for i in range(sz):
      rmap = []
      for row in range(0,self.dataset[i].shape[0],rstep):
        count = 0
        for col in range(self.dataset[i].shape[1]-1,-1,-1 * cstep):
          if self.dataset[i][row][col] == 0:
            count = count + 1
          else:
            break
        rmap.append(count)
      right_maps.append(rmap)
    return right_maps

  def top_maps(self, rstep, cstep):
    sz = len(self.dataset)
    top_maps = []
    for i in range(sz):
      tmap = []
      for col in range(0,self.dataset[i].shape[1],cstep):
        count = 0
        for row in range(0,self.dataset[i].shape[0], rstep):
          if self.dataset[i][row][col] == 0:
            count = count + 1
          else:
            break
        tmap.append(count)
      top_maps.append(tmap)
    return top_maps
  
  def bottom_maps(self, rstep, cstep):
    sz = len(self.dataset)
    b_maps = []
    for i in range(sz):
      bmap = []
      for col in range(0,self.dataset[i].shape[0],cstep):
        count = 0
        for row in range(self.dataset[i].shape[1]-1,-1,-1 * rstep):
          if self.dataset[i][row][col] == 0:
            count = count + 1
          else:
            break
        bmap.append(count)
      b_maps.append(bmap)
    return b_maps

In [8]:
### Testing
m = Data_Maps(train_arrays)

l_11 = m.left_maps(1,1)
r_11 = m.right_maps(1,1)
t_11 = m.top_maps(1,1)
b_11 = m.bottom_maps(1,1)

l_11d = m.map_diff(l_11)
r_11d = m.map_diff(r_11)
t_11d = m.map_diff(t_11)
b_11d = m.map_diff(b_11)

l_22 = m.left_maps(2,2)
r_22 = m.right_maps(2,2)
t_22 = m.top_maps(2,2)
b_22 = m.bottom_maps(2,2)

l_22d = m.map_diff(l_22)
r_22d = m.map_diff(r_22)
t_22d = m.map_diff(t_22)
b_22d = m.map_diff(b_22)

l_23 = m.left_maps(2,3)
r_23 = m.right_maps(2,3)
t_32 = m.top_maps(3,2)
b_32 = m.bottom_maps(3,2)

l_23d = m.map_diff(l_23)
r_23d = m.map_diff(r_23)
t_32d = m.map_diff(t_32)
b_32d = m.map_diff(b_32)

l_32 = m.left_maps(3,2)
r_32 = m.right_maps(3,2)
t_23 = m.top_maps(2,3)
b_23 = m.bottom_maps(2,3)

l_32d = m.map_diff(l_32)
r_32d = m.map_diff(r_32)
t_23d = m.map_diff(t_23)
b_23d = m.map_diff(b_23)

l_33 = m.left_maps(3,3)
r_33 = m.right_maps(3,3)
t_33 = m.top_maps(3,3)
b_33 = m.bottom_maps(3,3)

l_33d = m.map_diff(l_33)
r_33d = m.map_diff(r_33)
t_33d = m.map_diff(t_33)
b_33d = m.map_diff(b_33)


In [88]:
l_11d = torch.FloatTensor(l_11d) / torch.max(l_11d)
r_11d = torch.FloatTensor(r_11d) / torch.max(r_11d)
t_11d = torch.FloatTensor(t_11d) / torch.max(t_11d)
b_11d = torch.FloatTensor(b_11d) / torch.max(b_11d)

l_22d = torch.FloatTensor(l_22d) / torch.max(l_22d)
r_22d = torch.FloatTensor(r_22d) / torch.max(r_22d)
t_22d = torch.FloatTensor(t_22d) / torch.max(t_22d)
b_22d = torch.FloatTensor(b_22d) / torch.max(b_22d)

l_23d = torch.FloatTensor(l_23d) / torch.max(l_23d)
r_23d = torch.FloatTensor(r_23d) / torch.max(r_23d)
t_32d = torch.FloatTensor(t_32d) / torch.max(t_32d)
b_32d = torch.FloatTensor(b_32d) / torch.max(b_32d)

l_32d = torch.FloatTensor(l_32d) / torch.max(l_32d)
r_32d = torch.FloatTensor(r_32d) / torch.max(r_32d)
t_23d = torch.FloatTensor(t_23d) / torch.max(t_23d)
b_23d = torch.FloatTensor(b_23d) / torch.max(b_23d)

l_33d = torch.FloatTensor(l_33d) / torch.max(l_33d)
r_33d = torch.FloatTensor(r_33d) / torch.max(r_33d)
t_33d = torch.FloatTensor(t_33d) / torch.max(t_33d)
b_33d = torch.FloatTensor(b_33d) / torch.max(b_33d)

In [89]:
lefts = torch.cat([l_11d,l_22d,l_23d,l_32d,l_33d], dim = 1)
rights = torch.cat([r_11d,r_22d,r_23d,r_32d,r_33d], dim = 1)
tops = torch.cat([t_11d,t_22d,t_32d,t_23d,t_33d], dim = 1)
bottoms = torch.cat([b_11d,b_22d,b_32d,b_23d,b_33d], dim = 1)


print("Left shape = {}".format(lefts.shape))
print("Right shape = {}".format(rights.shape))
print("tops shape = {}".format(tops.shape))
print("bottom shape = {}".format(bottoms.shape))

batch_size = 200

batched_lefts = torch.split(lefts, batch_size)
batched_rights = torch.split(rights,batch_size)
batched_tops = torch.split(tops, batch_size)
batched_bottoms = torch.split(bottoms, batch_size)

train_labels = torch.FloatTensor(train_labels)
batched_labels = torch.split(train_labels, batch_size)

Left shape = torch.Size([60000, 76])
Right shape = torch.Size([60000, 76])
tops shape = torch.Size([60000, 76])
bottom shape = torch.Size([60000, 76])


In [90]:
batched_bottoms[0].shape

torch.Size([200, 76])

In [91]:
embedding = torch.cat([lefts, rights, tops, bottoms], dim =1)
batched_embeddings = torch.split(embedding,batch_size)

In [92]:
embedding.shape

torch.Size([60000, 304])

In [104]:
import torch.nn as nn
import torch.nn.functional as F

class Auto_encoder(nn.Module):
    def __init__(self, input_size, hidden_dims):
        super(Auto_encoder,self).__init__()
        ## Encoder part
        self.fclayer1 = nn.Linear(input_size, hidden_dims[0])
        torch.nn.init.xavier_uniform_(self.fclayer1.weight)
        self.fclayer2 = nn.Linear(hidden_dims[0], hidden_dims[1])
        torch.nn.init.xavier_uniform_(self.fclayer2.weight)
        
        ## Decoder part
        self.fclayer3 = nn.Linear(hidden_dims[1], hidden_dims[2])
        torch.nn.init.xavier_uniform_(self.fclayer3.weight)
        self.fclayer4 = nn.Linear(hidden_dims[2], input_size)
        torch.nn.init.xavier_uniform_(self.fclayer4.weight)
        
        
    def forward(self,x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded
    
    def encoder(self,x):
        z = self.fclayer1(x)
        z = F.relu(z)
        z = self.fclayer2(z)
        z = F.relu(z)
        return z
    
    def decoder(self,x):
        z = self.fclayer3(x)
        z = F.relu(z)
        z = self.fclayer4(z)
        z = F.tanh(z)
        return z
        
        

In [108]:
input_size  = 76
hidden_dims = [50, 7, 50]

L = Auto_encoder(input_size, hidden_dims).to(device)
R = Auto_encoder(input_size, hidden_dims).to(device)
T = Auto_encoder(input_size, hidden_dims).to(device)
B = Auto_encoder(input_size, hidden_dims).to(device)

lr = 0.005
criterion = nn.MSELoss()
left_optimizer = torch.optim.Adam(L.parameters(),lr)
right_optimizer = torch.optim.Adam(R.parameters(),lr)
top_optimizer = torch.optim.Adam(T.parameters(),lr)
bottom_optimizer = torch.optim.Adam(B.parameters(), lr)

optimizer = torch.optim.Adam([
                    {"params" : L.parameters()}, 
                    {"params" : R.parameters()},
                    {"params" : T.parameters()},
                    {"params" : B.parameters()}
                ], lr)

print("The number of parameters in the Left Auto Encoder -> {}".format(sum(p.numel() for p in L.parameters())))
print("The number of parameters in the Right Auto Encoder -> {}".format(sum(p.numel() for p in R.parameters())))
print("The number of parameters in the Top Auto Encoder -> {}".format(sum(p.numel() for p in T.parameters())))
print("The number of parameters in the Bottom Auto Encoder -> {}".format(sum(p.numel() for p in B.parameters())))



The number of parameters in the Left Auto Encoder -> 8483
The number of parameters in the Right Auto Encoder -> 8483
The number of parameters in the Top Auto Encoder -> 8483
The number of parameters in the Bottom Auto Encoder -> 8483


In [109]:
L

Auto_encoder(
  (fclayer1): Linear(in_features=76, out_features=50, bias=True)
  (fclayer2): Linear(in_features=50, out_features=7, bias=True)
  (fclayer3): Linear(in_features=7, out_features=50, bias=True)
  (fclayer4): Linear(in_features=50, out_features=76, bias=True)
)

In [110]:
EPOCHS = 50

for epoch in range(EPOCHS):
    print("Epoch-{}/{}---------->".format(epoch+1, EPOCHS))
    elloss = 0
    erloss = 0
    etloss = 0
    ebloss = 0
    for lbatch in batched_lefts:
        left_optimizer.zero_grad()
        left_output = L(lbatch.cuda())
        left_loss = criterion(lbatch.cuda(), left_output)
        left_loss.backward()
        left_optimizer.step()
        elloss += left_loss.item()
    print("Left Loss = {}".format(elloss))
    
    for rbatch in batched_rights:
        right_optimizer.zero_grad()
        right_output = R(rbatch.cuda())
        right_loss = criterion(rbatch.cuda(), right_output)
        right_loss.backward()
        right_optimizer.step()
        erloss += right_loss.item()
    print("Right Loss = {}".format(erloss))
    
    for tbatch in batched_tops:
        top_optimizer.zero_grad()
        top_output = T(tbatch.cuda())
        top_loss = criterion(tbatch.cuda(), top_output)
        top_loss.backward()
        top_optimizer.step()
        etloss += top_loss.item()
    print("Top Loss = {}".format(etloss))
    
    for bbatch in batched_bottoms:
        bottom_optimizer.zero_grad()
        bottom_output = B(bbatch.cuda())
        bottom_loss = criterion(bbatch.cuda(), bottom_output)
        bottom_loss.backward()
        bottom_optimizer.step()
        ebloss += bottom_loss.item()
    print("Bottom Loss = {}".format(ebloss))

Epoch-1/50---------->
Left Loss = 5.471939002163708
Right Loss = 4.938678524456918
Top Loss = 3.877004706300795
Bottom Loss = 5.072792822495103
Epoch-2/50---------->
Left Loss = 3.1906876377761364
Right Loss = 2.9287747549824417
Top Loss = 2.3226532679982483
Bottom Loss = 2.832639312837273
Epoch-3/50---------->
Left Loss = 2.8540559294633567
Right Loss = 2.750038967002183
Top Loss = 2.0145242433063686
Bottom Loss = 2.4968747128732502
Epoch-4/50---------->
Left Loss = 2.75388924498111
Right Loss = 2.6578700179234147
Top Loss = 1.7664566771127284
Bottom Loss = 2.377609960734844
Epoch-5/50---------->
Left Loss = 2.694149731192738
Right Loss = 2.439744924660772
Top Loss = 1.651797174476087
Bottom Loss = 2.306738579645753
Epoch-6/50---------->
Left Loss = 2.6435239957645535
Right Loss = 2.3308506100438535
Top Loss = 1.5948739643208683
Bottom Loss = 2.255140410270542
Epoch-7/50---------->
Left Loss = 2.604639384429902
Right Loss = 2.2874191752634943
Top Loss = 1.5582550284452736
Bottom Loss 

In [111]:
left_output[0]

tensor([ 0.0190,  0.0132, -0.0181, -0.0212,  0.5081,  0.0525,  0.0469,  0.0097,
         0.0398,  0.0362,  0.0195,  0.0191, -0.0399, -0.0519, -0.1291, -0.0875,
        -0.0030,  0.0532,  0.1317,  0.1115,  0.0554,  0.0182, -0.0474, -0.0240,
        -0.7011, -0.0168, -0.0077, -0.0182,  0.0214, -0.0160,  0.5168,  0.0625,
         0.0497,  0.0403, -0.0843, -0.2194,  0.0490,  0.2615,  0.0772, -0.0748,
        -0.7102, -0.0322, -0.0251,  0.0175,  0.4953,  0.0973,  0.0770,  0.0248,
        -0.1248, -0.2636,  0.0808,  0.2690,  0.1015, -0.1093, -0.6833,  0.0021,
         0.0061,  0.5138,  0.1001,  0.0624, -0.2230, -0.0389,  0.3197, -0.0559,
        -0.7251, -0.0068,  0.0026,  0.5011,  0.1561,  0.0535, -0.2841, -0.0401,
         0.3573, -0.0837, -0.6944, -0.0052], device='cuda:0',
       grad_fn=<SelectBackward0>)

In [99]:
lbatch[0]

tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.3214,  0.1786,  0.1071,  0.0000,
         0.0000,  0.0357,  0.0357,  0.0000,  0.0000, -0.2143, -0.0357,  0.0000,
         0.0000,  0.0000,  0.3571,  0.0000,  0.0000,  0.0000, -0.0357, -0.0714,
        -0.6786,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.5000,  0.0714,
         0.0714,  0.0000, -0.2143,  0.0000,  0.0000,  0.3571,  0.0000, -0.1429,
        -0.6429,  0.0000,  0.0000,  0.0000,  0.5000,  0.1000,  0.0000,  0.1000,
        -0.2000, -0.1000,  0.0000,  0.4000,  0.0000, -0.1000, -0.7000,  0.0000,
         0.0000,  0.5000,  0.0714,  0.0714, -0.2143,  0.0000,  0.3571, -0.1429,
        -0.6429,  0.0000,  0.0000,  0.5000,  0.1000,  0.1000, -0.3000,  0.0000,
         0.4000, -0.1000, -0.7000,  0.0000])